# Read and Write SEGY files with pysegy

This tutorial demonstrates how to read existing SEGY data shipped with the repository and how to write new files using **pysegy**. We use one of the example files located in the `data` directory.

In [ ]:
import sys
sys.path.append('..')  # allow importing the local pysegy package
import pysegy as seg
import fsspec
import matplotlib.pyplot as plt

## Read a SEGY file
We first load the file using `segy_read` without specifying a filesystem.

In [ ]:
path = '../data/overthrust_2D_shot_1_20.segy'
block = seg.segy_read(path)
print('Samples per trace:', block.fileheader.bfh.ns)
print('Number of traces:', len(block.traceheaders))

The first trace header contains a variety of fields. Here we display a few common values.

In [ ]:
hdr = block.traceheaders[0]
print('SourceX:', hdr.SourceX)
print('GroupX:', hdr.GroupX)
print('ns:', hdr.ns)

### Visualise a trace
The trace data are stored in a NumPy array. The following plot shows the amplitudes of the first trace.

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(block.data[:,0])
plt.title('Trace 0')
plt.xlabel('Sample')
plt.ylabel('Amplitude')
plt.tight_layout()
plt.show()

## Write a new file
We create a small subset containing the first 20 traces and write it to a temporary file.

In [ ]:
subset = seg.SeisBlock(block.fileheader, block.traceheaders[:20], block.data[:, :20])
out_path = 'subset.segy'
seg.segy_write(out_path, subset)
# reload to verify
out = seg.segy_read(out_path)
print('Wrote and reloaded', out_path, 'with', len(out.traceheaders), 'traces')

## Using fsspec filesystems
The same functions work with any filesystem that follows the `fsspec` interface. Here we use the local `file` implementation.

In [ ]:
fs = fsspec.filesystem('file')
block_fs = seg.segy_read(path, fs=fs)
print('Read with filesystem, ns =', block_fs.fileheader.bfh.ns)
fs_path = 'subset_fs.segy'
seg.segy_write(fs_path, subset, fs=fs)
reloaded = seg.segy_read(fs_path, fs=fs)
print('Reloaded via fs, traces =', len(reloaded.traceheaders))